In [1]:
from math import sqrt, acos, pi
from decimal import Decimal, getcontext

getcontext().prec = 30


class Vector(object):
    
    CANNOT_NORMALIZE_ZERO_VECTOR_MSG = 'Cannot normalize the zero vector'
    NO_UNIQUE_PARALLEL_COMPONENT_MSG = 'No Unique parallel component'
    NO_UNIQUE_ORTHOGONAL_COMPONENT_MSG = 'NO Unique Orthogonal component'

    def __init__(self, coordinates):
        try:
            if not coordinates:
                raise ValueError
            self.coordinates = tuple([Decimal(x) for x in coordinates])
            self.dimension = len(self.coordinates)

        except ValueError:
            raise ValueError('The coordinates must be nonempty')

        except TypeError:
            raise TypeError('The coordinates must be an iterable')

    def __str__(self):
        return 'Vector: {}'.format(self.coordinates)

    def __eq__(self, v):
        return self.coordinates == v.coordinates

    def magnitude(self):
        coordinates_squared = [x ** 2 for x in self.coordinates]
#         return sqrt(sum(coordinates_squared))
        return Decimal(sum(coordinates_squared)).sqrt()

    def normalised(self):
        try:
            magnitude = self.magnitude()
            return self.times_scalar(Decimal(1.0) / magnitude)

        except ZeroDivisionError:
            raise Exception('Cannot normalize the zero vector')

    def plus(self, v):
        #new_coordinates = [x + y for x, y in zip(self.coordinates, v.coordinates)]
        new_coordinates = []
        n = len(self.coordinates)
        for i in range(n):
            new_coordinates.append(self.coordinates[i] + v.coordinates[i])
        return Vector(new_coordinates)

    def minus(self, v):
        new_coordinates = [x - y for x, y in zip(self.coordinates, v.coordinates)]
        return Vector(new_coordinates)

    def times_scalar(self, c):
        new_coordinates = [Decimal(c) * x for x in self.coordinates]
        return Vector(new_coordinates)

    def dot(self, v):
        return sum([x * y for x, y in zip(self.coordinates, v.coordinates)])

    def angle_with(self, v, in_degrees=False):
        try:
            u1 = self.normalised()
            u2 = v.normalised()
            angle_in_radians = acos(u1.dot(u2))

            if in_degrees:
                degrees_per_radian = 180. / pi
                return angle_in_radians * degrees_per_radian
            else:
                return angle_in_radians

        except Exception as e:
            if str(e) == self.CANNOT_NORMALIZE_ZERO_VECTOR_MSG:
                raise Exception('Cannot comput an angle with a zero vector')
            else:
                raise e
                
    def is_orthogonal_to(self, v, tolerance=1e-10):
        return abs(self.dot(v)) < tolerance

    def is_parallel_to(self, v):
        return self.is_zero() or v.is_zero() or self.angle_with(v) == 0 or self.angle_with(v) == pi

    def is_zero(self, tolerance=1e-10):
        return self.magnitude() < tolerance
    
    def component_parallel_to(self, basis):
        try:
            u = basis.normalised()
            weight = self.dot(u)
            return u.times_scalar(weight)
        
        except Exception as e:
            if str(e) == self.CANNOT_NORMALIZE_ZERO_VECTOR_MSG:
                raise Exception(self.NO_UNIQUE_PARALLEL_COMPONENT_MSG)
            else:
                raise e
    
    def component_orthogonal_to(self, basis):
        try:
            projection = self.component_parallel_to(basis)
            return self.minus(projection)
        except Exception as e:
            if str(e) == self.NO_UNIQUE_PARALLEL_COMPONENT_MSG:
                raise Exception(self.NO_UNIQUE_ORTHOGONAL_COMPONENT_MSG)
            else:
                raise e
                
    def cross(self,v):
        try:
            x1,y1,z1 = self.coordinates
            x2,y2,z2 = v.coordinates
            new_coordinates = [y1*z2-y2*z1,-(x1*z2-x2*z1),x1*y2-x2*y1]
            return Vector(new_coordinates)
        
        except ValueError as e:
            msg = str(e)
            if msg == ('need more than 2 values to unpack'):
                self_make_R3 = Vector(self.coordinates + ('0',))
                v_make_R3 = Vector(v.coordinates + ('0',))
                return self_make_R3.cross(v_make_R3)
            
            elif(msg == 'too many values to unpack' or
                 msg == 'need more than 1 value to unpack'):
                raise Exception('Only defined in two dimension')
            else:
                raise e


    def area_of_parallelogram_with(self,v):
        cross_product = self.cross(v)
        return cross_product.magnitude()

    def area_of_triangle_with(self,v):
        return self.area_of_parallelogram_with(v)/Decimal('2.0')


In [2]:
my_vector = Vector([1,2,3])

In [3]:
print(my_vector)

Vector: (Decimal('1'), Decimal('2'), Decimal('3'))


In [4]:
my_vector2 = Vector([1,2,3])

In [5]:
my_vector3 = Vector([-1,2,3])

In [6]:
print(my_vector == my_vector2)

True


In [7]:
print(my_vector == my_vector3)

False


In [8]:
print(8.218 + -1.129)
print(-9.341 + 2.111)

7.089
-7.229999999999999


In [9]:
v = Vector([8.218, -9.341])
w = Vector([-1.129, 2.111])

print(v.plus(w))

Vector: (Decimal('7.088999999999999968025576891'), Decimal('-7.229999999999999094058011906'))


In [10]:
print(7.119 - -8.223)
print(8.215 - 0.878)

15.342
7.337


In [11]:
v = Vector([7.119, 8.215])
w = Vector([-8.223, 0.878])

print(v.minus(w))

Vector: (Decimal('15.34200000000000052580162446'), Decimal('7.336999999999999855226917589'))


In [12]:
print(7.41 * 1.671)
print(7.41 * -1.012)
print(7.41 * -0.318)

12.38211
-7.49892
-2.35638


In [13]:
c = 7.41
v = Vector([1.671, -1.012, -0.318])

print(v.times_scalar(c))

Vector: (Decimal('12.38211000000000054020787843'), Decimal('-7.498920000000000222790674798'), Decimal('-2.356380000000000081388229489'))


In [14]:
v = Vector([-0.221, 7.437])
print(v.magnitude())

7.440282924728064675549167963


In [15]:
v = Vector([8.813, -1.331, -6.247])
print(v.magnitude())

10.88418756729228776690849357


In [16]:
v = Vector([5.581, -2.136])
print(v.normalised())

Vector: (Decimal('0.9339352140866403209395391566'), Decimal('-0.3574423252623299934727680370'))


In [17]:
v = Vector([1.996, 3.108, -4.554])
print(v.normalised())

Vector: (Decimal('0.3404012959433013534467308533'), Decimal('0.5300437012984872951141974901'), Decimal('-0.7766470449528028348026506789'))


In [18]:
(7.887* -8.802) + (4.138 * 6.776)

-41.382286

In [19]:
v = Vector([7.887, 4.138])
w = Vector([-8.802, 6.776])

print(v.dot(w))

-41.38228599999999454398391663


In [20]:
(-5.955 * -4.496) + (-4.904 * -8.755) + (-1.874 * 7.103) 

56.397178000000004

In [21]:
v = Vector([-5.955, -4.904, -1.874])
w = Vector([-4.496, -8.755, 7.103])

print(v.dot(w))

56.39717800000000569975711073


In [22]:
v = Vector([3.183, -7.627])
w = Vector([-2.668, 5.3199])

print(v.angle_with(w))

3.0720941120523664


In [23]:
v = Vector([7.35, 0.221, 5.188])
w = Vector([2.751, 8.259, 3.985])

print(v.angle_with(w, in_degrees=True))

60.27581120523091


In [24]:
v = Vector([-7.579, -7.88])
w = Vector([22.737, 23.64])

print(v.is_orthogonal_to(w))
print(v.is_parallel_to(w))
print(v.is_zero())

False
True
False


In [25]:
v = Vector([-2.029, 9.97, 4.172])
w = Vector([-9.231, -6.639, -7.245])

print(v.is_orthogonal_to(w))
print(v.is_parallel_to(w))
print(v.is_zero())

False
False
False


In [26]:
v = Vector([-2.328, -7.284, -1.214])
w = Vector([-1.821, 1.072, -2.94])

print(v.is_orthogonal_to(w))
print(v.is_parallel_to(w))
print(v.is_zero())

True
False
False


In [27]:
v = Vector([2.118, 4.827])
w = Vector([0, 0])

print(v.is_orthogonal_to(w))
print(v.is_parallel_to(w))
print(v.is_zero())

True
True
False


In [28]:
v = Vector([3.039, 1.879])
w = Vector([0.825, 2.036])

print(v.component_parallel_to(w))

Vector: (Decimal('1.082606962484466633513478909'), Decimal('2.671742758325302083755623254'))


In [29]:
v = Vector([-9.88, -3.264, -8.159])
w = Vector([-2.155, -9.353, -9.473])

print(v.component_orthogonal_to(w))

Vector: (Decimal('-8.350081043195763114117174637'), Decimal('3.376061254287719853135464877'), Decimal('-1.433746042781185897982350548'))


In [30]:
v = Vector([3.009, -6.172, 3.692, -2.51])
w = Vector([6.404, -9.144, 2.759, 8.718])

print(v.component_parallel_to(w))
print(v.component_orthogonal_to(w))

Vector: (Decimal('1.968516167214089824655988641'), Decimal('-2.810760748439356316065414000'), Decimal('0.8480849633578503584828686371'), Decimal('2.679813233256157911289340035'))
Vector: (Decimal('1.040483832785910072315314674'), Decimal('-3.361239251560643392612064338'), Decimal('2.843915036642149812047387945'), Decimal('-5.189813233256157698126519307'))


In [31]:
v = Vector([8.462, 7.893, -8.187])
w = Vector([6.984, -5.975, 4.778])

print(v.cross(w))

Vector: (Decimal('-11.20457099999999773371683886'), Decimal('-97.60944399999999084633373058'), Decimal('-105.6851619999999939140451488'))


In [32]:
v = Vector([-8.987, -9.838, 5.031])
w = Vector([-4.268, -1.861, -8.866])

print(v.area_of_parallelogram_with(w))

142.1222214018463287494546634


In [33]:
v = Vector([1.5, 9.547, 3.691])
w = Vector([-6.007, 0.124, 5.772])

print(v.area_of_triangle_with(w))

42.56493739941893351623305821
